https://www.cnblogs.com/wj-1314/p/9579490.html

https://keras-cn.readthedocs.io/en/latest/getting_started/sequential_model/

https://keras.io/api/layers/recurrent_layers/lstm/

inputs: A 3D tensor with shape [batch, timesteps, feature].

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

#### 加载数据，打乱数据，切分数据，整理数据维度

In [2]:
# 路径
train_fn_0 = '../../dataset/train_0.csv'
train_fn_1 = '../../dataset/train_1.csv'
test_fn_0 = '../../dataset/test_0.csv'
test_fn_1 = '../../dataset/test_1.csv'
# 加载训练集
train_0 = np.loadtxt(train_fn_0,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
train_1 = np.loadtxt(train_fn_1,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
train = np.concatenate((train_0,train_1),axis=0)
np.random.shuffle(train)
# 加载测试集
test_0 = np.loadtxt(test_fn_0,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
test_1 = np.loadtxt(test_fn_1,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
test = np.concatenate((test_0, test_1), axis=0)
np.random.shuffle(test)
# 整理数据维度
x_train = train[:,0:30][:,:,np.newaxis]
x_test = test[:,0:30][:,:,np.newaxis]
y_train = train[:,30][:,np.newaxis]
y_test = test[:,30][:,np.newaxis]
print(x_train.shape)
print(y_train.shape)

(38022, 30, 1)
(38022, 1)


#### LSTM 

In [3]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', return_sequences=True, input_shape=(30, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(x_train, y_train,validation_split=0.3,batch_size=500, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=500)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            16896     
_________________________________________________________________
dropout (Dropout)            (None, 30, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 29,345
Trainable params: 29,345
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
54/54 [==============================] - 12s 227ms/step - loss: 0.5129 - accuracy: 0.7488 - val_loss: 0.316

In [4]:
from sklearn.metrics import classification_report
LSTM_y_pred=model.predict(x_test, batch_size=500)
print(classification_report(y_test, LSTM_y_pred.round(),digits=4))

              precision    recall  f1-score   support

         0.0     0.9918    0.9518    0.9714      6872
         1.0     0.8863    0.9795    0.9305      2633

    accuracy                         0.9595      9505
   macro avg     0.9390    0.9657    0.9510      9505
weighted avg     0.9626    0.9595    0.9601      9505



In [ ]:
from timeit import default_timer as timer
test_time_lstm = []
for i in range(10):
    start_time = timer()
    print(start_time)
    model.predict(x_test, batch_size=500)
    current_time = timer()
    print(current_time)
    test_time_lstm.append(current_time-start_time)
    print(current_time-start_time)
    i = i+1
print(test_time_lstm)
print(np.mean(test_time_lstm))

#### GRU

In [5]:
from keras.layers import GRU
from timeit import default_timer as timer

In [13]:
GRU_model = Sequential()
GRU_model.add(GRU(units=64, activation='relu', return_sequences=True, input_shape=(30, 1)))
GRU_model.add(Dropout(0.2))
GRU_model.add(GRU(units=32, activation='relu', return_sequences=False))
GRU_model.add(Dropout(0.2))
GRU_model.add(Dense(1, activation='sigmoid'))

GRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
GRU_model.summary()
start_time = timer()
GRU_model.fit(x_train, y_train, validation_split=0.3,batch_size=512, epochs=10)
current_time = timer()
print("train_time",(current_time-start_time)/10)
score = GRU_model.evaluate(x_test, y_test, batch_size=500)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 30, 64)            12864     
_________________________________________________________________
dropout_7 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
gru_6 (GRU)                  (None, 32)                9408      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 22,305
Trainable params: 22,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
54/54 [==============================] - 6s 117ms/step - loss: 0.5560 - accuracy: 0.7034 - val_loss: 0.42

In [24]:
GRU_model2 = Sequential()
GRU_model2.add(GRU(units=32, activation='relu', return_sequences=True, input_shape=(30, 1)))
GRU_model2.add(Dropout(0.1))
GRU_model2.add(GRU(units=16, activation='relu', return_sequences=False))
GRU_model2.add(Dropout(0.2))
GRU_model2.add(Dense(1, activation='sigmoid'))

GRU_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
GRU_model2.summary()
start_time = timer()
GRU_model2.fit(x_train, y_train, validation_split=0.3,batch_size=512, epochs=10)
current_time = timer()
print("train_time",(current_time-start_time)/10)
score = GRU_model2.evaluate(x_test, y_test, batch_size=500)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_15 (GRU)                 (None, 30, 32)            3360      
_________________________________________________________________
dropout_17 (Dropout)         (None, 30, 32)            0         
_________________________________________________________________
gru_16 (GRU)                 (None, 16)                2400      
_________________________________________________________________
dropout_18 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 5,777
Trainable params: 5,777
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
52/52 [==============================] - 5s 92ms/step - loss: 0.6028 - accuracy: 0.7200 - val_loss: 0.5171 

In [21]:
gru_y_pred=GRU_model2.predict(x_test, batch_size=500)
print(classification_report(y_test, gru_y_pred.round(),digits=4))

              precision    recall  f1-score   support

         0.0     0.9933    0.9748    0.9840      6872
         1.0     0.9373    0.9829    0.9596      2633

    accuracy                         0.9771      9505
   macro avg     0.9653    0.9789    0.9718      9505
weighted avg     0.9778    0.9771    0.9772      9505



In [15]:
test_time_gru = []
for i in range(10):
    start_time = timer()
    print(start_time)
    GRU_model.predict(x_test, batch_size=500)
    current_time = timer()
    print(current_time)
    test_time_gru.append(current_time-start_time)
    print(current_time-start_time)
    i = i+1
print(test_time_gru)
print(np.mean(test_time_gru))

48860473.81530041
48860474.45369187
0.6383914574980736
48860474.45401027
48860475.038261525
0.5842512547969818
48860475.03844524
48860475.604981475
0.5665362328290939
48860475.60530355
48860476.168778285
0.5634747371077538
48860476.16886417
48860476.73980388
0.5709397122263908
48860476.740209565
48860477.30245292
0.5622433573007584
48860477.30254658
48860477.896065325
0.5935187414288521
48860477.89615407
48860478.4791888
0.5830347314476967
48860478.479277074
48860479.059711955
0.5804348811507225
48860479.059803106
48860479.648323335
0.5885202288627625
[0.6383914574980736, 0.5842512547969818, 0.5665362328290939, 0.5634747371077538, 0.5709397122263908, 0.5622433573007584, 0.5935187414288521, 0.5830347314476967, 0.5804348811507225, 0.5885202288627625]
0.5831345334649086


#### BiGRU

In [8]:
from keras.layers import Bidirectional

In [9]:
BiGRU_model = Sequential()
BiGRU_model.add(Bidirectional(GRU(64), input_shape=(30, 1)))
BiGRU_model.add(Dropout(0.2))
BiGRU_model.add(Dense(1, activation='sigmoid'))
BiGRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
BiGRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
BiGRU_model.summary()

BiGRU_model.fit(x_train, y_train, validation_split=0.3,batch_size=500, epochs=10)
score = BiGRU_model.evaluate(x_test, y_test, batch_size=500)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 128)               25728     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 25,857
Trainable params: 25,857
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
54/54 [==============================] - 7s 123ms/step - loss: 0.4837 - accuracy: 0.7507 - val_loss: 0.3012 - val_accuracy: 0.8810
Epoch 2/10
54/54 [==============================] - 6s 113ms/step - loss: 0.2669 - accuracy: 0.8944 - val_loss: 0.2389 - val_accuracy: 0.9040
Epoch 3/10
54/54 [==============================] - 6s 109ms/step - loss: 0.2328 - accuracy: 0.9

In [16]:
BiGRU_y_pred=BiGRU_model.predict(x_test, batch_size=500)
print(classification_report(y_test, BiGRU_y_pred.round(),digits=4))

              precision    recall  f1-score   support

         0.0     0.9776    0.9897    0.9836      6872
         1.0     0.9721    0.9408    0.9562      2633

    accuracy                         0.9761      9505
   macro avg     0.9749    0.9652    0.9699      9505
weighted avg     0.9761    0.9761    0.9760      9505

